In [1]:
from datetime import datetime, timezone, timedelta
import numbers
import itertools
from collections import namedtuple


Confirmation = namedtuple("Confirmation", "account_number transaction_code transaction_id time_utc time")

class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes):
        if name is None or len(str(name).strip()) == 0:
            raise ValueError("Timezone name cannot be empty.")
        self._name = str(name).strip()

        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError("Hour offset must be an integer")
        
        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError("Minute offset must be an integer.")

        if offset_minutes > 59 or offset_minutes < -59:
            raise ValueError("Minutes offset must be between -59 and 59 (inclusive).")
        
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError("Offset must be between -12:00 and +14:00.")
        
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset

    @property
    def name(self):
        return self._name

    def __eq__(self, other):
        return(isinstance(other, TimeZone) and 
                self.name == other.name and 
                self._offset_hours == other._offset_hours and
                self._offset_minutes == other._offset_minutes)

    def __repr__(self):
        return (f"TimeZone(name='{self._name}' "
                f"offset_hours={self._offset_hours}, "
                f"offset_minutes={self._offset_minutes})")


tz1 = TimeZone("ABC", -2, -15)

In [2]:
tz1.offset

datetime.timedelta(days=-1, seconds=78300)

In [40]:
class Account:

    _interest_rate = 0.05
    transaction_counter = itertools.count(0)
    account_nums = []
    _transaction_codes = {
        "deposit": "D",
        "withdraw": "W",
        "interest": "I",
        "rejected": "X"
    }
    
    def __init__(self, account_number, first_name, last_name, timezone=None, initial_balance=0,):
        # if not isinstance(account_number, numbers.Integral):
        #     raise ValueError("Account number must be an integer.")
        # global account_nums
        # if account_num in account_nums:
        #     raise ValueError(f"The account with number {account_num} already exists.")
        self._account_number = account_number
        # account_nums.append(account_num)

        if not isinstance(first_name, str) or len(str(first_name).strip()) == 0:
            raise ValueError(f"Please provide a valid first name.")
        self._first_name = first_name

        if not isinstance(last_name, str) or len(str(last_name).strip()) == 0:
            raise ValueError(f"Please provide a valid first name.")
        self._last_name = last_name
        
        if timezone is None:
            timezone = TimeZone("UTC", 0, 0)
        self.timezone = timezone

        self._balance = Account.validate_real_number(initial_balance, min_value=0)

    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self):
        return self._first_name
    
    @property
    def last_name(self):
        return self._last_name

    @property
    def full_name(self):
        return f"{self._first_name} {self._last_name}"
    
    @property
    def timezone(self):
        return self._timezone
    
    @property
    def balance(self):
        return self._balance
    
    @first_name.setter
    def first_name(self, new_name):
        old_name = self._first_name
        self.validate_and_set_name("_first_name", new_name, "first name")
        print(f"Successfully changed First Name from: {old_name} to {self._first_name}")

    @last_name.setter
    def last_name(self, new_name):
        old_name = self.last_name
        self.validate_and_set_name("_last_name", new_name, "last name")
        print(f"Successfully changed First Name from: {old_name} to {self.last_name}")
    
    @timezone.setter
    def timezone(self, new_timezone):
        if not isinstance(new_timezone, TimeZone):
            raise ValueError("Time zone must be a valid timezone object.")
        self._timezone = new_timezone
        print(f"Successfully changed the timezone to the following: {self.timezone}")


    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate 
    
    @classmethod
    def set_interest_rate(cls, new_interest_rate):
        if not isinstance(new_interest_rate, numbers.Real):
            raise ValueError("Interest rate must be a real number.")
        if new_interest_rate < 0:
            raise ValueError("Interest rate cannot be negative.")
        cls._interest_rate = new_interest_rate
        print(f"Successfully updated interest rate to {new_interest_rate}")

    def validate_and_set_name(self, attr_name, value, field_title):
        if not isinstance(value, str) or len(str(value).strip()) == 0:
            raise ValueError(f"Please provide a valid {field_title}.")
        setattr(self, attr_name, value)
    


    def generate_confirmation_code(self, transaction_code):
        transaction_datetime = datetime.now(timezone.utc).strftime("%Y%m%d%H%M%S")
        return (f"{transaction_code}-"
                f"{self.account_number}-"
                f"{transaction_datetime}-"
                f"{next(Account.transaction_counter)}")


    def deposit(self, deposit_value):
        deposit_value = Account.validate_real_number(deposit_value, 0.01)
        transaction_code = Account._transaction_codes["deposit"]
        conf_code = self.generate_confirmation_code(transaction_code)
        self._balance += deposit_value
        return conf_code



    def withdraw(self, withdrawal_value):
        withdrawal_value = Account.validate_real_number(withdrawal_value, 0.01)
        accepted = False
        if self.balance - withdrawal_value < 0: 
            transaction_code = Account._transaction_codes["rejected"]
        else:
            accepted = True
            transaction_code = Account._transaction_codes["withdraw"]

        conf_code = self.generate_confirmation_code(transaction_code)
        if accepted:
            self._balance -= withdrawal_value
        return conf_code


    def pay_interest(self):
        interest_amount = self.balance * Account.get_interest_rate()
        conf_code = self.generate_confirmation_code(self._transaction_codes["interest"])
        self._balance += interest_amount
        return conf_code
    

    @staticmethod
    def validate_real_number(value, min_value=None):
        if not isinstance(value, numbers.Real):
            raise ValueError("Value must be a real number.")
        if min_value is not None and value < min_value:
            raise ValueError(f"Value must be at least {min_value}")
        return value


    @staticmethod
    def parse_confirmation_code(confirmation_code, preferred_time_zone=None):
        parts = confirmation_code.split("-")
        if len(parts) != 4:
            raise ValueError("Invalid confirmation code")
        transaction_code, account_number, raw_dt_utc, transaction_id = parts

        try:
            dt_utc = datetime.strptime(raw_dt_utc, "%Y%m%d%H%M%S")
        except ValueError as ex:
            raise ValueError("Invalid transaction datetime") from ex
        
        if preferred_time_zone is None:
            preferred_time_zone = TimeZone("UTC", 0, 0)
        
        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError("Invalid TimeZone specified.")
        
        dt_preferred = dt_utc + preferred_time_zone.offset
        dt_preferred_str = f"{dt_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({preferred_time_zone.name})"

        return Confirmation(account_number, 
                            transaction_code, 
                            transaction_id, 
                            dt_utc.isoformat(), 
                            dt_preferred_str)


    def __repr__(self):
        return (f"Account Number: {self.account_number}\n"
                f"First Name: {self.first_name}\n"
                f"Last Name: {self.last_name}\n"
                f"Timezone: {self.timezone}\n"
                f"Balance: ${self.balance}")


a = Account(account_number="A100", 
                 first_name="Eric", 
                 last_name="Idle",
                 initial_balance=100,
                 timezone=TimeZone("MST ", -7, 0))


print(a.balance)
print(a.deposit(150.02))
print(a.balance)
print(a.withdraw(0.02))
print(a.balance)
Account.set_interest_rate(0.01)
print(a.get_interest_rate())
print(a.pay_interest())
print(a.balance)


Successfully changed the timezone to the following: TimeZone(name='MST' offset_hours=-7, offset_minutes=0)
100
D-A100-20250306095423-0
250.02
W-A100-20250306095423-1
250.0
Successfully updated interest rate to 0.01
0.01
I-A100-20250306095423-2
252.5


In [41]:
import unittest

def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)


class TestAccount(unittest.TestCase):
    def setUp(self):
        print("running setup...")
        self.x = 100

    def tearDown(self):
        print("running teardown...")


    def test_ok(self):
        self.x = 200
        self.assertEqual(100, self.x)

    def test_2(self):
        self.assertEqual(100, self.x)

run_tests(TestAccount)

test_2 (__main__.TestAccount.test_2) ... ok
test_ok (__main__.TestAccount.test_ok) ... FAIL

FAIL: test_ok (__main__.TestAccount.test_ok)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\veace\AppData\Local\Temp\ipykernel_21240\672938005.py", line 20, in test_ok
    self.assertEqual(100, self.x)
AssertionError: 100 != 200

----------------------------------------------------------------------
Ran 2 tests in 0.003s

FAILED (failures=1)


running setup...
running teardown...
running setup...
running teardown...


In [44]:
class TestAccount(unittest.TestCase):
    def setUp(self):
        self.account_number = "A100"
        self.first_name = "FIRST"
        self.last_name = "LAST"
        self.tz = TimeZone("TZ", 1, 30)
        self.balance = 100.00

    def create_account(self):
        return Account(self.account_number, self.first_name, self.last_name, self.tz, self.balance)


    def test_create_timezone(self):
        tz = TimeZone("ABC", -1, -30)
        self.assertEqual("ABC", tz.name)
        self.assertEqual(timedelta(hours=-1, minutes=-30), tz.offset)
    
    def test_timezone_equal(self):
        tz1 = TimeZone("ABC", -1, -30)
        tz2 = TimeZone("ABC", -1, -30)
        self.assertEqual(tz1, tz2)
    
    def test_timezone_not_equal(self):
        tz = TimeZone("ABC", -1, -30)
        test_timezones = (
            TimeZone("DEF", -1, -30),
            TimeZone("ABC", -1, 0),
            TimeZone("ABC", 1, -30),
            # TimeZone("ABC", -1, -30)
        )

        for i, test_tz in enumerate(test_timezones):
            with self.subTest(test_number=f"Test # {i}"):
                self.assertNotEqual(tz, test_tz)


    def test_create_account(self):

        a = self.create_account()

        self.assertEqual(self.account_number, a.account_number)
        self.assertEqual(self.first_name, a.first_name)
        self.assertEqual(self.last_name, a.last_name)
        self.assertEqual(self.first_name + " " + self.last_name, a.full_name)
        self.assertEqual(self.tz, a.timezone)
        self.assertEqual(self.balance, a.balance)


    def test_create_account_blank_first_name(self):
        self.first_name = ""

        with self.assertRaises(ValueError):
            a = self.create_account()
            
    def test_create_account_negative_balance(self):
        self.balance = -100.00
    
        with self.assertRaises(ValueError):
            a = self.create_account()
            
    def test_account_withdraw_ok(self):
        account_number = "A100"
        first_name = "FIRST"
        last_name = "LAST"
        tz = TimeZone("TZ", 1, 30)
        balance = 100.00
        withdrawal_amount = 20

        a = Account(account_number=account_number,
                        first_name=first_name,
                        last_name=last_name,
                        initial_balance=balance)
        
        conf_code = a.withdraw(withdrawal_amount)
        self.assertTrue(conf_code.startswith("W-"))
        self.assertEqual(balance - withdrawal_amount, a.balance)

    def test_account_withdraw_overdraw(self):
        account_number = "A100"
        first_name = "FIRST"
        last_name = "LAST"
        tz = TimeZone("TZ", 1, 30) 
        balance = 100.00
        withdrawal_amount = 200

        a = Account(account_number=account_number,
                        first_name=first_name,
                        last_name=last_name,
                        initial_balance=balance)
        
        conf_code = a.withdraw(withdrawal_amount)
        self.assertTrue(conf_code.startswith("X-"))
        self.assertEqual(balance, a.balance)


    
run_tests(TestAccount)

test_account_withdraw_ok (__main__.TestAccount.test_account_withdraw_ok) ... ok
test_account_withdraw_overdraw (__main__.TestAccount.test_account_withdraw_overdraw) ... ok
test_create_account (__main__.TestAccount.test_create_account) ... ok
test_create_account_blank_first_name (__main__.TestAccount.test_create_account_blank_first_name) ... ok
test_create_account_negative_balance (__main__.TestAccount.test_create_account_negative_balance) ... ok
test_create_timezone (__main__.TestAccount.test_create_timezone) ... ok
test_timezone_equal (__main__.TestAccount.test_timezone_equal) ... ok
test_timezone_not_equal (__main__.TestAccount.test_timezone_not_equal) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.015s

OK


Successfully changed the timezone to the following: TimeZone(name='UTC' offset_hours=0, offset_minutes=0)
Successfully changed the timezone to the following: TimeZone(name='UTC' offset_hours=0, offset_minutes=0)
Successfully changed the timezone to the following: TimeZone(name='TZ' offset_hours=1, offset_minutes=30)
Successfully changed the timezone to the following: TimeZone(name='TZ' offset_hours=1, offset_minutes=30)
